# Install dependencies
- commented out as libraries already installed

In [ ]:
#!conda install -y nb_conda
#!conda install -y -c conda-forge census
#!pip install us

# Import libraries

In [ ]:
import pandas as pd
import numpy as np

# Read Diabetes observation table

In [ ]:
df_obs = pd.read_csv('synthea_diabetes_obs.csv')
df_obs

# Read diabetes population

In [ ]:
df_pop = pd.read_csv('pop_a1c.csv')
df_pop

# Read denominator/demographics for all patients

In [ ]:
df_patients = pd.read_csv('synthea_allpatients.csv',dtype = {'zipcode': str},parse_dates=['birthDate'])
print(df_patients.dtypes)
df_patients

# Use only latest observation for Hemoglobin A1c measure

In [ ]:
latest_obs = df_obs[df_obs['latest'] == 1]
latest_obs

# Create numerator (patients with latest A1c observation >= 9)

In [ ]:
numerator_obs = latest_obs.loc[latest_obs['obs_value'] >= 9.,'patient_id']
numerator_obs

In [ ]:
numerator = df_pop[df_pop['patient_id'].isin(numerator_obs.values)]
numerator = numerator.drop_duplicates(subset=['patient_id'])
numerator

# Create denominator (all patients with diabetes, 18-75 in age)

In [ ]:
denominator = df_pop[df_pop['patient_id'].isin(latest_obs['patient_id'].unique())]
denominator = denominator.drop_duplicates(subset=['patient_id'])
denominator

# Create target 
* 1 = uncontrolled diabates or A1c >= 9
* 0 = controlled diabetes

In [ ]:
df = denominator.copy()
df

In [ ]:
df['target'] = 0
df.loc[df['patient_id'].isin(numerator['patient_id'].values),'target'] = 1
df

# Merge patient information and save for visualization

In [ ]:
df = pd.merge(df,df_patients,left_on='patient_id',right_on='id',how='left')

In [ ]:
df.to_csv('tmp.datastudio.a1c.csv',index=False)

# Create census data/Social determinants of Health table 
- Example: poverty rate
- see https://api.census.gov/data/2019/acs/acs5/variables.html for variables
  * B01003_001E - total population
  * B17015_001E - total - poverty status
  * B17015_002E - total income in past 12 months below poverty level

In [ ]:
from census import Census
from us import states

API_KEY = 'API_KEY'  # replace with own API_KEY; see census API website to obtain API key 
c = Census(API_KEY)

In [ ]:
data = c.acs5.get(('NAME','B01003_001E','B17015_001E','B17015_002E'), {'for': 'zip code tabulation area:*', 'in' : 'state:25'})
census_df = pd.DataFrame.from_dict(data)
census_df

In [ ]:
# Rename columns

In [ ]:
census_df.columns = ['NAME','Population','Poulation Below Poverty Line','Population Below Poverty Line past 12 months','state','Zip Code']
census_df

# Create Poverty precentage

In [ ]:
census_df['Poverty Percentage'] = census_df['Poulation Below Poverty Line']/census_df['Population']
census_df

# Select columns

In [ ]:
census_df = census_df[['Zip Code','Population', 'Poulation Below Poverty Line',
       'Population Below Poverty Line past 12 months','Poverty Percentage']]
census_df

# Save census/sdoh table

In [ ]:
census_df.to_csv('census_poverty.csv',header=True,index=False)